In [2]:
import psycopg2

conn = psycopg2.connect(
    dbname="ragdb",
    user="postgres",
    password="1234",
    host="localhost",
    port="5432"
)

cur = conn.cursor()

In [31]:
import pandas as pd

df = pd.read_csv("/home/ys0660/25-1/Demo/data/details_2025-05-22.csv")  # 파일 경로에 맞게 수정

In [40]:
from datetime import datetime

# 날짜 문자열을 datetime 객체로 변환
df["start_date"] = df["start_date"].where(df["start_date"].notna(), None)
df["end_date"] = df["end_date"].where(df["end_date"].notna(), None)

In [53]:
cur.execute("""
    SELECT column_name
    FROM information_schema.columns
    WHERE table_name = 'festivals'
    ORDER BY ordinal_position;
""")
columns = cur.fetchall()
for col in columns:
    print(col[0])


id
name
location
start_date
end_date
description
homepage
contents
poster
updated_at


In [46]:
columns_to_drop = [
    "host", "organizer", "address_road", "address_lot",
    "latitude", "longitude", "created_at", "region"
]

# 각 컬럼에 대해 DROP COLUMN 실행
for col in columns_to_drop:
    try:
        cur.execute(f"ALTER TABLE festivals DROP COLUMN IF EXISTS {col};")
        print(f"Dropped column: {col}")
    except Exception as e:
        print(f"Error dropping column {col}: {e}")
        conn.rollback()
    else:
        conn.commit()

Dropped column: host
Dropped column: organizer
Dropped column: address_road
Dropped column: address_lot
Dropped column: latitude
Dropped column: longitude
Dropped column: created_at
Dropped column: region


In [43]:
for _, row in df.iterrows():
    try:
        cur.execute("""
            INSERT INTO festivals (
                name, location, start_date, end_date,
                description, contents, homepage, poster
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            row["name"],
            row["location"],
            row["start_date"],
            row["end_date"],
            row["description"],
            row["contents"],
            row["homepage"],
            row["poster"]
        ))
    except Exception as e:
        print(f"Error inserting row: {row['name']}, error: {e}")
        conn.rollback()  # 이 행의 실패로 인한 트랜잭션 초기화

In [55]:
cur.execute("SELECT * FROM festivals LIMIT 5;")
rows = cur.fetchall()
for row in rows:
    print(row)

(3230, '단양 소백산 철쭉제', '충청북도 단양군 단양읍 수변로 38', datetime.date(2025, 5, 22), datetime.date(2025, 5, 25), '단양 소백산철쭉제는 매 년 5월에 개최된다. 5월 소백산 정상에 핀 철쭉을 즐기며 산행을 체험하고 단양강을 따라 펼쳐지는 수변특설무대를 배경으로 소백산 산신제, 실버가요제, 개막축하공연, 레이져 쇼, EDM공연, 서커스공연을 즐길 수 있으며 상상의거리 일대에선 철쭉과 관련된 체험행사와 버스킹공연 등이 펼쳐진다. 풍물시장과 지역에서 유명한 먹거리들을 한 자리에서 만끽 할 수 있는 즐거운 축제다.', 'https://korean.visitkorea.or.kr/kfes/detail/fstvlDetail.do?fstvlCntntsId=0adb8f34-4b47-4eee-99f6-f64dc6717c68&cntntsNm=%EB%8B%A8%EC%96%91%EC%86%8C%EB%B0%B1%EC%82%B0%EC%B2%A0%EC%AD%89%EC%A0%9C', '1.공연프로그램: 불꽃을 활용한 레이져쇼,동춘서커스, 개막콘서트(이찬원, 클라씨,왁스,김지훈), 철쭉하모니 콘서트(송가인, 복지은,판타지보이스, 해바라기), 제8회 실버가요제2. 부대프로그램 : 퇴계이황 추념 서예대회, 소백산 산신제,모터서프 축하공연,단양사투리 경연대회, 철쭉야생화 테마관, 버스킹공연 등3. 소비자 참여 프로그램 : 철쭉을 활용한 체험부스, 재미철철 감성쭉쭉 먹거리 체험(전통주,이색먹거리)4. 기타 내용 : 철쭉포토죤에서 인생 샷 만나기, 단양10경 플리마켓, 단양빵지순례(단양에서 유명한 빵집이 마련한 철쭉빵) 거리에서 펼쳐지는 재미철철 감성쭉쭉(뮤지컬, 마임, 마당극 공연), 풍물시장 등', 'https://kfescdn.visitkorea.or.kr/kfes/upload/contents/db/0adb8f34-4b47-4eee-99f6-f64dc6717c68_3.jpg', datetime.datetime(2025, 5, 22,

In [3]:
cur.execute("SELECT COUNT(*) FROM festivals;")
count = cur.fetchone()[0]
print(f"총 데이터 수: {count}행")

총 데이터 수: 1239행


In [4]:
# 직접 하나만 넣고 싶을 때
single_data = {
    "name": "부산국제록페스티벌",
    "location": "부산광역시 사상구 삼락동 29-50",
    "start_date": "2025-09-26",
    "end_date": "2025-09-28",
    "description": """*하기 축제 먹거리 정보는 전년도(2024년) 정보로, 2025년도 축제 먹거리 정보는 업데이트 중에 있습니다.*

    부산국제록페스티벌은 2000년 첫 개최 이래, 26년 동안 명맥을 꾸준히 지켜온 국내 최초·최장수 국제록페스티벌이다. 2025년에는 9월 26일(금)부터 9월 28일(일)까지 3일간, 삼락생태공원에서 개최되며, 다양한 국내외 아티스트가 참여할 예정이다.공원 곳곳에 조성된 무대에서 신진 아티스트부터 글로벌 최정상급 헤드라이너까지 취향에 따라 다양한 공연을 관람할 수 있으며,관람객 참여형 게임 이벤트, 푸드코트 라운지, 캠핑존 등 다양한 부대 이벤트도 즐길 수 있다.""",
    "contents": "1. 메인프로그램 : 부산국제록페스티벌\n2. 부대프로그램 : Rookies on the BU-ROCK, Road to BU-ROCK 등\n3. 기타 : MD부스, 푸드코트 라운지, 캠핑존 등",
    "homepage": "https://korean.visitkorea.or.kr/kfes/detail/fstvlDetail.do?fstvlCntntsId=8146cbf1-2012-4178-aa12-06ddfb702361&cntntsNm=%EB%B6%80%EC%82%B0%EA%B5%AD%EC%A0%9C%EB%A1%9D%ED%8E%98%EC%8A%A4%ED%8B%B0%EB%B2%8C",
    "poster": "https://kfescdn.visitkorea.or.kr/kfes/upload/contents/db/8146cbf1-2012-4178-aa12-06ddfb702361_3.png"
}


try:
    cur.execute("""
        INSERT INTO festivals (
            name, location, start_date, end_date,
            description, contents, homepage, poster
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        single_data["name"],
        single_data["location"],
        single_data["start_date"],
        single_data["end_date"],
        single_data["description"],
        single_data["contents"],
        single_data["homepage"],
        single_data["poster"]
    ))
    conn.commit()  # 성공하면 커밋
except Exception as e:
    print(f"Error inserting data: {single_data['name']}, error: {e}")
    conn.rollback()  # 실패 시 롤백

cur.execute("UPDATE festivals SET updated_at = NOW();")

In [5]:
conn.commit()
cur.close()
conn.close()